In [9]:
import pandas as pd
import json
from py2neo import Graph, Node, Relationship
import numpy as np
import time
from functools import reduce
from tqdm.autonotebook import tqdm

/opt/conda/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [ ]:
graph = Graph("bolt://neo4j-magtwo:7687", auth=('neo4j','myneo'))

n_nodes = graph.database.primitive_counts['NumberOfNodeIdsInUse']
n_relationships = graph.database.primitive_counts['NumberOfRelationshipIdsInUse']
print("Connected to graph database with {:,} nodes and {:,} relationships!".format
     (n_nodes, n_relationships))

In [8]:
for years_post_pub in range(4):
    query = """
    CALL apoc.periodic.iterate("
    MATCH (q:Quanta)-[:PUBLISHED_IN]->(y:Year)
    WITH DISTINCT q, y.value as year 
    MATCH (p:Quanta)-[:CITES]->(q:Quanta) 
    WHERE p.year = year+{}
    WITH DISTINCT p, q, year
    RETURN [p,q,year] as l
    ","
    WITH l[0] AS p, l[1] AS q, l[2] as year
    MATCH (a:Author)-[:AUTHORED]->(p) 
    WITH q, year+{} as p1, apoc.coll.toSet(collect(a.name)) AS alist 
    MERGE (q)-[m1:METRICS_IN]->(a:Year {{value: p1}})
    SET m1.earlyAdopters = size(alist)
    ",{{batchSize:5000, iterateList:true, parallel:false}});
    """.format(years_post_pub, years_post_pub)
    print(query)


    CALL apoc.periodic.iterate("
    MATCH (q:Quanta)-[:PUBLISHED_IN]->(y:Year)
    WITH DISTINCT q, y.value as year 
    MATCH (p:Quanta)-[:CITES]->(q:Quanta) 
    WHERE p.year = year+0
    WITH DISTINCT p, q, year
    RETURN [p,q,year] as l
    ","
    WITH l[0] AS p, l[1] AS q, l[2] as year
    MATCH (a:Author)-[:AUTHORED]->(p) 
    WITH q, year+0 as p1, apoc.coll.toSet(collect(a.name)) AS alist 
    MERGE (q)-[m1:METRICS_IN]->(a:Year {value: p1})
    SET m1.earlyAdopters = size(alist)
    ",{batchSize:5000, iterateList:true, parallel:false});
    

    CALL apoc.periodic.iterate("
    MATCH (q:Quanta)-[:PUBLISHED_IN]->(y:Year)
    WITH DISTINCT q, y.value as year 
    MATCH (p:Quanta)-[:CITES]->(q:Quanta) 
    WHERE p.year = year+1
    WITH DISTINCT p, q, year
    RETURN [p,q,year] as l
    ","
    WITH l[0] AS p, l[1] AS q, l[2] as year
    MATCH (a:Author)-[:AUTHORED]->(p) 
    WITH q, year+1 as p1, apoc.coll.toSet(collect(a.name)) AS alist 
    MERGE (q)-[m1:METRICS_IN]->(a:Yea